# setup

```bash
cd E:\programas\ia\virtual_environment && my_env_3129\Scripts\activate
uv pip install -qU langchain faiss-cpu pypdf2 openai python-dotenv langchain-ollama
```

In [ ]:
PATH_ARQUIVOS = r"../../data"
LANG = "por" # por eng
persist_directory = "../../db/faiss_db"  # Diretório onde o banco de dados será salvo

## Recuperar FAISS

In [ ]:
import os
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_ollama import OllamaEmbeddings

embedding_model = OllamaEmbeddings(model="nomic-embed-text") # nomic-embed-text | llama3

def get_vector_store():
    index_file = os.path.join(persist_directory, "index.faiss") # Nome padrão do arquivo de índice FAISS
    if os.path.exists(index_file): # Verifica se o arquivo de índice FAISS existe
        return FAISS.load_local(persist_directory, embedding_model, allow_dangerous_deserialization=True)
    else:
        # 1. Obtenha a dimensão do embedding
        example_embedding = embedding_model.embed_query("texto de exemplo") # Use um texto de exemplo para obter um embedding
        embedding_dimension = len(example_embedding)

        # 2. Crie um índice FAISS vazio com a dimensão correta
        faiss_index = faiss.IndexFlatL2(embedding_dimension) # IndexFlatL2 é um índice FAISS comum

        # 3. Inicialize FAISS da Langchain com o índice FAISS vazio
        vectorstore = FAISS(
            embedding_function=embedding_model,  # 1. embedding_function
            index=faiss_index,                 # 2. index
            docstore=InMemoryDocstore(),       # 3. docstore (using InMemoryDocstore for clarity)
            index_to_docstore_id={},          # 4. index_to_docstore_id
        )
        vectorstore.save_local(persist_directory)
        # print(f"Nova base FAISS vazia criada e salva em: {persist_directory}")
        return vectorstore
        
# vectorstore = get_vector_store()

### testes

In [25]:
from uuid import uuid4
from langchain_core.documents import Document

vectorstore = get_vector_store()

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

documents = [
    document_1,
    document_2
]
uuids = ["my_id_001", "my_id_002"]

vectorstore.add_documents(documents=documents, ids=uuids)

['my_id_001', 'my_id_002']

In [40]:
id_aux = uuids[1]
results = vectorstore.get_by_ids([id_aux])

print(results[0])
print(results[0].id)
print(id_aux in results[0].id)
#print(results['ids'] and [uuids[1]] in results['ids'])

page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.' metadata={'source': 'news'}
my_id_002
True


## torch

In [18]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 4090
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


## docling

In [19]:
from docling.datamodel.base_models import InputFormat
from docling_core.types.doc import ImageRefMode
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode, EasyOcrOptions, TesseractOcrOptions, OcrMacOptions
from docling.datamodel.settings import settings

IMAGE_RESOLUTION_SCALE = 2.0

# Define pipeline options for PDF processing
pipeline_options = PdfPipelineOptions(
    do_table_structure=True,  # Enable table structure detection
    do_ocr=True,  # Enable OCR
    # full page ocr and language selection
    #ocr_options=EasyOcrOptions(force_full_page_ocr=True, lang=["en"]),  # Use EasyOCR for OCR
    ocr_options=TesseractOcrOptions(force_full_page_ocr=True, lang=[LANG]),  # Uncomment to use Tesseract for OCR
    #ocr_options = OcrMacOptions(force_full_page_ocr=True, lang=['en-US']),
    table_structure_options=dict(
        do_cell_matching=False,  # Use text cells predicted from table structure model
        mode=TableFormerMode.ACCURATE  # Use more accurate TableFormer model
    ),
    generate_page_images=True,  # Enable page image generation
    generate_picture_images=True,  # Enable picture image generation
    images_scale=IMAGE_RESOLUTION_SCALE, # Set image resolution scale (scale=1 corresponds to a standard 72 DPI image)
)

# Initialize the DocumentConverter with the specified pipeline options
doc_converter_global = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

### Chunking (estudo)

#### direct chunking

In [ ]:
result = doc_converter_global.convert(Path(r"data\pdfs\monopoly.pdf"))
display(result.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED))

In [ ]:
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker

#conv_res = DocumentConverter().convert(FIRST_10_PAGES)
#doc = conv_res.document

chunker = HybridChunker(tokenizer="BAAI/bge-small-en-v1.5")  # set tokenizer as needed
chunk_iter = chunker.chunk(result.document)

# Convert the iterator to a list to count the chunks
chunks = list(chunk_iter)
num_chunks = len(chunks)

# Print the number of chunks
print(f"The document has been divided into {num_chunks} chunks.")

In [ ]:
print(chunks[1].text)

#### faiss chunking

obs: precisa do ollama executando `ollama serve`

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [ ]:
# display(result.document.export_to_markdown())
from langchain_core.documents import Document

local_path = r"data\pdfs\monopoly.pdf"
result = doc_converter_global.convert(Path(local_path))
documento = Document(page_content=result.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED), metadata={"source": local_path})

print(documento)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents([documento])

In [ ]:
print(f"num chunks: {len(chunks)}")
print(chunks[0])

# Separar documentos

In [21]:
import os

def separar_arquivos(diretorio):
    """
    Varre um diretório e suas subpastas, separando arquivos de imagem de outros tipos de arquivo.

    Args:
        diretorio (str): O caminho do diretório a ser varrido.

    Returns:
        tuple: Uma tupla contendo duas listas: imagens e documentos.
    """

    imagens = []
    documentos = []

    extensoes_imagens = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Adicione outras extensões se necessário

    for raiz, subpastas, arquivos in os.walk(diretorio):
        for arquivo in arquivos:
            caminho_arquivo = os.path.join(raiz, arquivo)
            nome_arquivo, extensao = os.path.splitext(arquivo)
            extensao = extensao.lower()

            if extensao in extensoes_imagens:
                imagens.append(caminho_arquivo) #adiciona o caminho completo
            else:
                documentos.append(caminho_arquivo) #adiciona o caminho completo

    return imagens, documentos

imagens, documentos = separar_arquivos(PATH_ARQUIVOS)

# print("Imagens:")
# for imagem in imagens: print(imagem)

# print("\nDocumentos:")
# for documento in documentos: print(documento)

# Document chunks

In [23]:
from pathlib import Path
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# display(result.document.export_to_markdown())
from langchain_core.documents import Document

# img
import pytesseract
from PIL import Image
#pytesseract.pytesseract.tesseract_cmd = r"E:\programas\ia\Tesseract-OCR\tesseract.exe"
#end img

text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)

#local_path = r"data\pdfs\monopoly.pdf"
def get_chunks_doc(local_path):
    result = doc_converter_global.convert(Path(local_path))
    documento = Document(page_content=result.document.export_to_markdown(image_mode=ImageRefMode.EMBEDDED), metadata={"source": local_path})
    chunks = text_splitter.split_documents([documento])
    return chunks


def get_chunks_image(local_path):
    image = Image.open(local_path)
    extracted_text = pytesseract.image_to_string(image, lang=LANG)

    documento = Document(page_content=extracted_text, metadata={"source": local_path})
    chunks = text_splitter.split_documents([documento])
    return chunks

# FAISS db

obs: precisa do ollama executando `ollama serve`

## faiss batch

In [ ]:
import hashlib
import os
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
#from langchain_community.embeddings import OllamaEmbeddings
# from langchain_ollama import OllamaEmbeddings 
# from pathlib import Path

def generate_id_filename(filename, page_index):
    filename = os.path.basename(filename)
    base_id = hashlib.sha256(filename.encode()).hexdigest()
    return f"{base_id}_{page_index}"    

def generate_id(document, page_index):
    """Gera um ID único baseado no nome do arquivo e no índice da página."""
    source = document.metadata['source']
    return generate_id_filename(os.path.basename(source), page_index)


# embedding_model = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = get_vector_store()

def faiss_indexing(
    path_arquivos=PATH_ARQUIVOS,
    vectorstore=vectorstore
):
    """Indexa chunks em lote no FAISS."""

    imagens, documentos = separar_arquivos(path_arquivos)

    for imagem in imagens:
        id_aux = generate_id_filename(imagem, 0)
        results = vectorstore.get_by_ids([id_aux]);
        if results and results[0].id and id_aux in results[0].id:
            print(f"Documento com ID {id_aux} | {os.path.basename(imagem)} já existe na coleção.")
            continue

        faiss_indexing_batch(get_chunks_image(imagem), vectorstore)

    for documento in documentos:
        id_aux = generate_id_filename(documento, 0)
        results = vectorstore.get_by_ids([id_aux]);
        if results and results[0].id and id_aux in results[0].id:
            print(f"Documento com ID {id_aux} | {os.path.basename(documento)} já existe na coleção.")
            continue

        faiss_indexing_batch(get_chunks_doc(documento), vectorstore)

    return vectorstore
    

def faiss_indexing_batch(chunks, vectorstore=None):
    """Indexa chunks em lote no FAISS db."""

    if not chunks or not vectorstore:
        print('Sem chunks e/ou vectorstore is null')
        return

    documents_to_add = []
    ids_to_add = []
    #embeddings_to_add = []
    metadatas_to_add = []

    for i, chunk in enumerate(chunks):
        document_id = generate_id(chunk, i)
        results = vectorstore.get_by_ids([document_id]);
        if results and results[0].id and document_id in results[0].id:
            print(f"Documento com ID {document_id} já existe na coleção.")
            continue

        #embedding = embedding_model.embed_documents([chunk.page_content])[0]

        documents_to_add.append(chunk)
        ids_to_add.append(document_id)
        #embeddings_to_add.append(embedding)
        #metadatas_to_add.append(chunk.metadata)

    if documents_to_add:
        # , metadatas=metadatas_to_add
        vectorstore.add_documents(documents=documents_to_add, ids=ids_to_add)
        print(f"Adicionados {len(documents_to_add)} documentos em lote.")

    if chunks:
        first_chunk_id = generate_id(chunks[0], 0)
        print(f"ID do primeiro chunk: {first_chunk_id}")


## FAISS indexing

In [57]:
vectorstore = faiss_indexing(PATH_ARQUIVOS, vectorstore)
# salvar no disco
vectorstore.save_local(persist_directory)

Documento com ID ff355d2b2b6da4cf0f4fff0d3a48f8d7bf920e8e9fd5854d743d94b8d74431cb_0 | Captura de tela 2025-03-13 085540.png já existe na coleção.
Documento com ID 4de61c5de3846ef479128ce449eca7c0e6bacbb02259525bd3120ed54b454ead_0 | word.docx já existe na coleção.
Documento com ID f2a73aa28943b31bbc82bf3d4bfade03f80a0a6f5db83cfe1bfcb498aae83ff1_0 | xlsx.xlsx já existe na coleção.
Documento com ID 20c21a195a1b20b321c6807cfa29d8f02e7c362e80828dbbf5449bd8259c8255_0 | python - How to run DeepSeek model locally - Stack Overflow.html já existe na coleção.
Documento com ID 7c0100cbcab341da2d7e3063ea01534e24cd669c505f7845f3ccc8bd95ddba50_0 | LLMs.pdf já existe na coleção.
Documento com ID 35ee8ca3231e1a9d9ea39b17b42b8c0fc604929073ef313ce42096db30ea7847_0 | monopoly.pdf já existe na coleção.
Documento com ID a8f083315429e2f971a0292de1197fef3a7059aa7ef6a1698b1c1d2c0775341e_0 | scammer-agent.pdf já existe na coleção.
Documento com ID 27c24834bbaf23c523d8d555b4026d3e51710cc16d86a1d1eb55092a793fcbaa

In [58]:
query_generica = "documento"  # Uma consulta genérica
results = vectorstore.similarity_search(query_generica, k=5) # k=5 para obter 5 resultados

for doc in results:
    print(doc)

page_content='- Automático
- Claro
- Noite' metadata={'source': '../../data\\docs\\word.docx'}
page_content='4vjKy8tramrYbLa7uztJkkVFRefPn4+MjFSpVKampsbGxtXV1ZWVlXQ63c3N7flS2EkkEhiG6+rqVCqVh4cHiuouvKioqKamprKystUvY7lcjuO4XC6vq6tzd3cH5y0tLa2rq3NycjI1NSUIorKyEvTczc0NVMgSi8VFRUWZmZlPUaSVl5dXV1dbWlra29uDilqlpaVMJtPNzQ10FZT6UiqVUqnUwsJCJpOdOnVq0KBBdnZ2oAUYQXCNBm6+CoA4J0deXi4tKSFwnCRJSV6eorqaaWxs7OOj00IVFxcePSrw9oZpNKapKcfKSlFdLS0shGFY6OPD+KvMRQu0MlljejqMokJvbzqX6zFhAolh0sJC8FeNWEyoVL6zZvGbQylJglA3NMAoKs7NZZub811cCK1WWlzsOmoU39X177uPYaK0NEypFHh6skxNtTKZOC8Pk0p5jo48R0cai9XiLC0AZ0HodHFeHtPExKi5ZWV1teSva6Hz+dKioqLjx+l8PkMgYBgZcW1t5VVVAjc3Y29v9K/JQ2g0orQ0EsdNAgJoLJZKJNLV9ygpgWk0Yx8fdWNjzYMH+QcPcqyt2RYWXHt7lMP51/EhSVKcna2sqWFbWhq5uytrahhCoW47QSiqqphCIcrlQghCNOc9Anh4ePTs2fP48eOLFi3SZ8R+VhgMBpjPLcjOzi4pKTEzMwsMDAQ7YBiWlpZWXV0tEAgCAwO5XF1tH41Gk5SUpNVq5XI5m81uEeelVqtTU1MbGhrMzMwCAgKYTGZlZWVTUxOfz8/OzjY1NQ0KCgK5AMRicXJyMpfLJQiixbXU19dXVFSYmJhkZ2dbWlr6+/vDMFxZWVlfX08QhFwuDwkJQVE0OTm5sbHRxcUFPAjFxcXqZn1

# Retrieval

In [59]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [60]:
# LLM from Ollama
local_model = "llama3.2"
local_model = "deepseek-r1"
llm = ChatOllama(model=local_model)

In [61]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Você é um assistente de modelo de linguagem de IA. Sua tarefa é gerar cinco
    versões diferentes da pergunta do usuário fornecida para recuperar documentos relevantes de
    um banco de dados vetorial. Ao gerar múltiplas perspectivas sobre a pergunta do usuário, seu
    objetivo é ajudar o usuário a superar algumas das limitações da pesquisa de similaridade 
    baseada em distância. Forneça essas perguntas alternativas separadas por quebras de linha.
    Pergunta original: {question}""",
)

In [ ]:
#from langchain_ollama import OllamaEmbeddings 
#embedding_model = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = get_vector_store()

retriever = MultiQueryRetriever.from_llm(
    vectorstore.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Responda à pergunta com base SOMENTE no seguinte contexto:
{context}
Pergunta: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [63]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [64]:
from IPython.display import display, Markdown
display(Markdown(chain.invoke("Como jogar Monopoly ?")))

<think>
Okay, eu preciso entender como se joga Monopóly. Parece que é um jogo de tabuleiro com players que se movem aoredo do tabuleiro, possuem casas e podem adquirir propriedades ou payas taxas. Eu agradeço a explicar cada passo e as regras básicas.
</think>

Jogar Monopóly é um jogo clássico de tabuleiro que combina strategies de negociação, finanças e lógica. Aqui está uma explicação passo a passo do como se jogar:

### **Regras Básicas**

1. **Objetivo**  
   O objetivo é ser o primeiro jogador a cumprir 40 casas do tabuleiro.

2. **Setup**  
   - Cada jogador escolhe uma propriedade de Monopóly como casa inicial.
   - Cada jogador recebe 20 dólares como capital inicial.
   - Os movimentos são feitos com um par dedados (duas moedas ou um cubo de dados).

3. **Passos Iniciais**  
   - **Primeiro Jogador**: lança os dados e move suas casas de acordo com o número total dos pontos.
   - **Segundo Jogador**: lança os dados após a primeira vez.

### **Jogando**

4. **Passar por "Go"**  
   Cada vez que um jogador passa por "Go", o Banca paga $200 para cada circulação completa.

5. **Propriedades e Casas**  
   - As casas são classificadas em:
     - **Vagas Sem Propriedade**: apenas se movimentam.
     - **Casa de Monopóly (com ou sem propriedade)**: podem ser adquiridas com dinheiro.
     - **Propriedades Comerciais (Comércio ou Loja de Roupas, por exemplo)**: podem ser adquiridas e vendidas.
     - **Casa de Ocupação**: onde um jogador está.

6. **Ad quarições e Penaltis**  
   - **Sorteios (Chance) e Condições de Comunidades (Community Chest)**:
     - Jogadores podem receber regras adicionais, como ter que pagar taxas, juros ou even realinear (ter que sair de casa).
   - **Dobradouro**: Se um jogador lança dobros (ambos os dados iguais), ele pode:
     - Mover duas vezes no mesmo movimento.
     -Cancelar um sorteeiro ou confeitório.
     - Tripularem: dois jouvendo tripulsar juntos.
     - Tripular: cada jogador lança dobros separadamente.

7. **Jogar Tripulando**  
   Se três ou mais jogadores lançam dobros simultaneamente, cada jogador tripula: todas as movimentações são feitas simultaneamente.

8. **Saindo de Casa**  
   - Se um jogador der servei de "Jail" (prisão), ele paga uma taxa fixa (ex: $50) para sair.
   - Cansado de Jaul: Se um jogador estiver em Jaul por mais de 3 vezes consecutivas, ele pago uma taxa anual (ex: $200).

9. **Acordar Dinheiro**  
   - Jogadores que adquirem propriedades podem cobrar de outros jogadores se estiverem vindo para a mesma casa.
   - Se um jogador der em torno de uma "Propriedade Comercial", ele pode negociar o preço de compra e venda.

10. **Paying Taxes**  
    - Cada propriedade tem associated taxas: como impostos de terreno, impostos de desconto (community tax) e impostos de juros.

### **Vencedor**

- O vencedor é aquele que primeiro cumprir 40 casas do tabuleiro.
- Se houver empate em torno de 40, os jogadores restantes passam por "Go" e o vencedor é aquele que terminou mais perto.

### **Recursos**

- **Casa de Monopóly**: Cada jogador recebe uma casa como propriedade inicial.
- **Dinheiro Inicial**: Cada jogador recebe $200 para cada circulação de "Go".
- **Propriedades**: Adquiridas com dinheiro, podem ser vendidas em qualquer momento.

### **Regras Adicionais**

- **Jail**: Uma casa de Ocupação que não pode ser vendida.
- **Incaralho (Chance) e Condições de Comunidades (Community Chest)**: Sorteios que podem incluir aquisição de propriedades adicionais, juros ou pagamentos.

Estas são as regras básicas do jogo Monopóly. O jogo combina movimentos viajados, negociação e finanças, tornando-se uma experiência divertida e estrategica!

In [65]:
display(Markdown(chain.invoke("Explique um Voice-Enabled AI e qual a conclusão dos autores ?")))

<think>
Primeiro, levi o texto fornecido sobre as linguagens de programação do Word e como a frase "A QUICK BROWN FOX JUMPS OVER THE LAZY DOG" é utilizada em diferentes plataformas. Isso me induziu a pensar em como os Sw Permission ou outros softwares usam tais pangramas para testar efeitos visuais de diferentes fontes.

Em seguida, acreditei que o autor estivesse falando sobre a importância de usar pangramas em português para evitar conflitos ao invés de usá-los automaticamente como na Inglês. Coisa que pareceu confuso, mas entendi que ele wanted para destacar a necessidade de ajustes locais em diferentes linguagens.

Então, me orientei para entender melhor o que é um Voice-Enabled AI e sua importancia. Sabia que os AIs como Siri ou Alexa podem interpretar e responder em voz humana, mas questionei se isso estava diretamente relacionado com os pangramas do Word.

Tive uma breve pesquisa sobre os Processos de Translação de Softwares e como os Sw Permission evitam os conflitos de linguagem. Depois, percebi que o autor talvez estivesse fechando um tópico diferente, talvez sobre a necessidade de ajustar os softwares para diferentesidiomas em lugar de falar sobre Voice-Enabled AI diretamente.

Assim, decidi me orientar em torno do AIVoice. Sabia que existem vários AIs para interfaces Virtuais como Siri, Amazon Lex, Google Assistant e outros. Entendi que o AIVoice visa tornar interação com os computadores mais fácil e natural, usando a voz humana.

No entanto, o texto não mencionou diretamente a relação entre os pangramas do Word e os Voice-Enabled AI. achei que o autor talvez estivesse abordando um problema diferente, talvez sobre como os softwares podem ser melhorados para usarem mais as interfaces de voz.

Finalmente, conclui que para esclarecer a relação entre os pangramas do Word e os Voice-Enabled AI, seria necessário mais informações ou contextos sobre o tema. Assim, decidi formular uma resposta baseando solely no que estava disponível.
</think>

**Resposta:**

Um Voice-Enabled AI é uma tecnologia de interação que permite ao usuário commands emitidos por voz para serem compreendidos e executados por um computador, como os smartphones, tablets ou computadores. esses AIs (Artificial Intelligence) são usados para tarefas como beber, pedir auxílio, levar direção ou performar tarefas básicas, tornando a interação humana-computador mais natural e intuitiva.

No contexto do texto mencionado, não há uma direta conexão entre os pangramas do Microsoft Word e os Voice-Enabled AI. entretanto, a idéia de ajustar as interfaces para diferentes idiomas e as necessidades do usuário pode ser relevantemente relacionada. Os autores podem estar destacando a necessidade de softwares serem adaptados para diferentesidiomas e usárias para evitar conflitos e melhorar a experiência de interação humana-computador. Assim, o uso de AIs como Voice-Enabled AI pode ajudar na definição de padrões ou interfaces mais personalizados que atendam as expectativas do usuário em diferentes linguagens e contextos.

In [66]:
display(Markdown(chain.invoke("Como executar o DeepSeek com python local ?")))

<think>
Ok, eu preciso entender como executar o DeepSeek com Python local. Primeiro, entendo que DeepSeek é uma plataforma que ajuda a gerar código para tarefas de dados clínicos usando LLMs. Eu Sou iniciante, então vou começar olhando para as instruções detalhadas e vejo que há vários passos a serem seguidos.

Primeiro, preciso instalar o Python no meu computador. Isso é tranquilo, posso baixar do site oficial. Depois, etes tarefas de instalação do Jupyter Notebook. Eu sou familiar com ele porque usando para machine learning, mas está a ser usado aqui para geração de código.

Agora, leio sobre o Repositório GitHub do DeepSeek. Preciso criar uma conta e downloadar os arquivos necessários. Acho que isso inclui o código-fonte da plataforma e talvez algum exemplo de como usar.

Em seguida, monto a GPU no computador para otimizar o desempenho. Eu sou incerto sobre como isso funciona exatamente, mas posso seguir as instruções passo a passo.

Depois disso, abro o Jupyter Notebook e digitro o comando para executar DeepSeek. Parece que é um comando simples, mas preciso garantir que tenho todas as dependências instaladas, como transformers e torch, antes de execute-lo.

No final, verifico os resultados na tela do Jupyter Notebook e escrevo minhas observações no arquivo PDF incluído. Isso me ajuda a manter um registro das coisas que fiz e como o DeepSeek funcionou para mim.

Enquanto isso, eu tiro as pernas à再去, tentando entender cada passo e resolver possíveis erros ao longo do processo.
</think>

Para executar o DeepSeek com Python local, siga os seguintes passos:

1. **Instalar o Python**: Baixe e instale a versão do Python desejada do site oficial.

2. **Baixar os Repositórios**: Crie uma conta no GitHub (se for necessária) e downloadar os arquivos necessários, incluindo o código-fonte da plataforma.

3. **Montar a GPU**: Se desejar accelerar o processamento, monte a GPU no seu computador.

4. **Iniciar o Jupyter Notebook**: Abra o arquivo `notebook.py` contido no Repositório do DeepSeek.

5. **Executar DeepSeek**:
   ```python
   from deepseek import DeepSeek

   # Inicialize o DeepSeek com sua configuração
   ds = DeepSeek(
       model='deepseek-chat',
       temperature=0.7,
       max_tokens=2048,
       n_ctx=2048,
       suffix="python"
   )

   # Execute uma tarefa de geração de código
   print("Meu comando:")
   print(ds.generate_code("Como executar o DeepSeek com python local?"))
   ```

6. **Visualizar os Resultados**: As saídas do código serán exibidas na tela do Jupyter Notebook.

**Observações**:
- **Configurações**: Ajuste os parâmetros como `temperature`, `max_tokens` e `n_ctx` conforme necessário para sua tarefa específica.
- **Arquivos Adicionais**: Se necessário, adicione as pastas e arquivos necessários no Repositório do DeepSeek.
- **Verificação de Erros**: Verifique os erros na tela, se houverem, e ajuste as configurações ou código correspondente.

Geralmente, o DeepSeek gera código que pode ser executado localmente após a instalação adequada das dependências.

# Excluir Faiss

In [ ]:
index_file = os.path.join(persist_directory, "index.faiss") # Nome padrão do arquivo de índice FAISS
if os.path.exists(index_file):
    os.remove(index_file)

index_pkl = os.path.join(persist_directory, "index.pkl") # Nome padrão do arquivo de pkl FAISS
if os.path.exists(index_pkl):
    os.remove(index_pkl)